In [130]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime

In [101]:
page_address_list=["https://www.immoregion.fr/vente/maison/metz/id-6708871.html","https://www.immoregion.fr/vente/maison/metz/id-6708870.html","https://www.immoregion.fr/vente/maison/saint-julien-les-metz/id-6696596.html","https://www.immoregion.fr/vente/maison/metz/id-6616097.html"]

In [172]:
json_list=[]
for link in page_address_list:
    page=requests.get(link)
    if page.status_code==200:
        page_content=BeautifulSoup(page.content,"html.parser")

        price=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[0].find_all("div")[0].text
        area=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[3].find_all("div")[0].text
        furnished=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[2].find_all("div")[0].text
        integral_area=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[4].find_all("div")[0].text
        nmbr_of_rooms=page_content.find_all("section",class_="feature sc-7vp35h-2-section-LayoutTheme__KeyGeneral-hbgJJa haNoV")[0].find_all("div", class_="feature-bloc-content-specification-content-response")[5].find_all("div")[0].text
        ref=page_content.find_all("h1",class_="KeyInfoBlockStyle__PdpTitle-sc-1o1h56e-2 hWEtva")[0].find_all("span")[0].text
        description=page_content.find_all("div",class_="DescriptionStyle__DescriptionContentDropdown-dqxmgt-3 dyQEIL")[0].find_all("div")[0].text
        link_address=link
        scrapping_time=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        agency=page_content.find_all("h3",class_="FormAgencyInfoStyle__AgencyName-sc-10hhmvq-5 jfZIgK")[0].text
        address=page_content.find_all("div",class_="block-localisation-address")[0].text

        json_data={"Ref_no":ref,"Price":price,"Surface":area,"furnished":furnished,"integral_area":whole_area,"No_room":nmbr_of_rooms,"Description":description,"Link":link_address,"scrapping_time":scrapping_time,"agency":agency,"address":address}
        json_list.append(json_data)
    else:
        pass

In [176]:
import pandas as pd
pd.DataFrame(json_list)

,Ref_no,Price,Surface,furnished,integral_area,No_room,Description,Link,scrapping_time,agency,address
0,Réf 6708871,310 000 €,"88,65 m²",Non,110 m²,2,Ce bien se trouve dans ce LotissementMetz Val...,https://www.immoregion.fr/vente/maison/metz/id...,2020-05-26 14:12:33,Laforet - Metz,rue Jeanne Jugan - Metz
1,Réf 6708870,420 000 €,"125,45 m²",Non,110 m²,4,Ce bien se trouve dans ce LotissementMetz Val...,https://www.immoregion.fr/vente/maison/metz/id...,2020-05-26 14:12:33,Laforet - Metz,rue Jeanne Jugan - Metz
2,Réf 6696596,307 000 €,Détails,NC,110 m²,5,"SAINT JULIEN LES METZ: Sur les hauteurs, pavil...",https://www.immoregion.fr/vente/maison/saint-j...,2020-05-26 14:12:34,DELAVILLE IMMOBILIER,57070 Saint-Julien-lès-MetzL'adresse exacte d...
3,Réf 6616097,299 900 €,110 m²,NC,110 m²,6,Maison Metz Queuleu :\nCharmante maison mitoye...,https://www.immoregion.fr/vente/maison/metz/id...,2020-05-26 14:12:34,Centrum Immobilier,57000 MetzL'adresse exacte de ce bien n'est p...


In [ ]:
"/vente/maison/metz/id-5824233.html"

In [216]:
import re
page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')})[-10:]

[<a href="/vente/maison/woippy/id-6692472.html"><div class="similar-add-slider-content-picture"><picture><img src="https://i1.static.athome.eu/images/annonces2/image_/87/e1/66/81cddbf7c6a325f66a4357aca72f33dc826090f8_244x160.jpg"/></picture></div><div class="similar-add-slider-content-info"><div class="similar-add-slider-content-name">Maison à Vendre</div><div class="similar-add-slider-content-info-content"><div class="similar-add-slider-content-city">Woippy</div><div class="similar-add-slider-container-price"><div class="similar-add-slider-content-price">295 400 €</div></div></div></div></a>,
 <a href="/vente/maison/metz/id-6672122.html"><div class="similar-add-slider-content-picture"><picture><img src="https://i1.static.athome.eu/images/annonces2/image_/af/ec/56/5c61f0addefe2855c75a8a5b28e4d14df8491841_244x160.jpg"/></picture></div><div class="similar-add-slider-content-info"><div class="similar-add-slider-content-name">Maison à Vendre</div><div class="similar-add-slider-content-info

In [245]:
seed_address="https://www.immoregion.fr/vente/maison/metz/id-6708871.html"
seed_address_list=[]

for i in range(0,4,1):
    if len(seed_address_list)==0:
        #add original seed_address
        seed_address_list.append(seed_address) 
        #read the original seed address and if reading process successful load html in var pagecontent
        page=requests.get(link)                 
        if page.status_code==200:
            page_content=BeautifulSoup(page.content,"html.parser")
            #iterate over 10 found links to other offers
            for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')})[-10:]:
                new_link="https://www.immoregion.fr"+link["href"]
                seed_address_list.append(link)
                
            counter=1
        else:
            counter=1        
    else:
        for link in seed_address_list[counter:]:
            page=requests.get(link)                 
            if page.status_code==200:
                new_links=[]
                page_content=BeautifulSoup(page.content,"html.parser")
                #iterate over 10 found links to other offers
                for link in page_content.find_all("a",{"href" : re.compile('/vente/maison/*/')})[-10:]:
                    new_link="https://www.immoregion.fr"+link["href"]
                    new_links.append(link)
                [seed_address_list.append(address) for address in new_links]
                counter+=10
            else:
                pass


SyntaxError: invalid syntax (<ipython-input-245-00b3cdf4ae2a>, line 28)

'/vente/maison/woippy/id-6692472.html'

In [238]:
seed_address_list=[]

True